In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression, Lasso, LassoCV

from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import xgboost as xgb

from copy import deepcopy

sns.set()
%matplotlib inline

/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
data_df = pd.read_csv('trimmed_cleaned_data.csv')
data_df['Top_1'] = data_df['Top_1'] * 100
print(data_df.shape)
data_df.head()

(2970, 29)


,Country,Year,Top_1,Tax_Rev_to_GDP,Individual_Spending_Rate,Society_Spending_Rate,Top_Rate,GDP_growth,1100,1200,...,4500,4600,5100,5110,5111,5120,5121,5200,5300,6000
0,Argentina,1965,NaN,NaN,NaN,NaN,NaN,10.569433,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Argentina,1966,NaN,NaN,NaN,NaN,NaN,-0.659726,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Argentina,1967,NaN,NaN,NaN,NaN,NaN,3.191997,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Argentina,1968,NaN,NaN,NaN,NaN,NaN,4.822501,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Argentina,1969,NaN,NaN,NaN,NaN,NaN,9.679526,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
data_df.dropna(subset=['Top_1'], inplace=True)
data_df.drop(['GDP_growth'], axis=1, inplace=True)
data_df.dropna(thresh=4, inplace=True)
data_df.reset_index(drop=True, inplace=True)
print(data_df.shape)
data_df.head()

(925, 28)


,Country,Year,Top_1,Tax_Rev_to_GDP,Individual_Spending_Rate,Society_Spending_Rate,Top_Rate,1100,1200,1300,...,4500,4600,5100,5110,5111,5120,5121,5200,5300,6000
0,Australia,1965,6.34,20.620,NaN,NaN,NaN,7.096,3.357,0.0,...,0.0,0.0,6.189,1.519,0.0,4.671,3.205,0.969,0.0,0.0
1,Australia,1966,6.16,19.840,NaN,NaN,NaN,7.072,2.971,0.0,...,0.0,0.0,5.894,1.403,0.0,4.492,3.107,0.946,0.0,0.0
2,Australia,1967,6.25,20.419,NaN,NaN,NaN,7.452,2.946,0.0,...,0.0,0.0,5.985,1.429,0.0,4.557,3.059,0.966,0.0,0.0
3,Australia,1968,6.05,20.349,NaN,NaN,NaN,7.262,3.177,0.0,...,0.0,0.0,5.875,1.509,0.0,4.366,2.881,0.956,0.0,0.0
4,Australia,1969,5.92,20.691,NaN,NaN,NaN,7.741,3.246,0.0,...,0.0,0.0,5.748,1.543,0.0,4.205,2.654,0.911,0.0,0.0


In [16]:
X = data_df.drop(['Country', 'Year', '1100', 'Top_1', '6000'], axis=1)
X.head()

,Tax_Rev_to_GDP,Individual_Spending_Rate,Society_Spending_Rate,Top_Rate,1200,1300,2100,2200,2300,2400,...,4400,4500,4600,5100,5110,5111,5120,5121,5200,5300
0,20.620,NaN,NaN,NaN,3.357,0.0,0.0,0.0,0.0,0.0,...,0.402,0.0,0.0,6.189,1.519,0.0,4.671,3.205,0.969,0.0
1,19.840,NaN,NaN,NaN,2.971,0.0,0.0,0.0,0.0,0.0,...,0.401,0.0,0.0,5.894,1.403,0.0,4.492,3.107,0.946,0.0
2,20.419,NaN,NaN,NaN,2.946,0.0,0.0,0.0,0.0,0.0,...,0.445,0.0,0.0,5.985,1.429,0.0,4.557,3.059,0.966,0.0
3,20.349,NaN,NaN,NaN,3.177,0.0,0.0,0.0,0.0,0.0,...,0.535,0.0,0.0,5.875,1.509,0.0,4.366,2.881,0.956,0.0
4,20.691,NaN,NaN,NaN,3.246,0.0,0.0,0.0,0.0,0.0,...,0.607,0.0,0.0,5.748,1.543,0.0,4.205,2.654,0.911,0.0


In [18]:
y = data_df.Top_1
y.head()

0    6.34
1    6.16
2    6.25
3    6.05
4    5.92
Name: Top_1, dtype: float64

In [21]:
data_lr = data_df.dropna()
print(data_lr.shape)
data_lr.head()

(289, 28)


,Country,Year,Top_1,Tax_Rev_to_GDP,Individual_Spending_Rate,Society_Spending_Rate,Top_Rate,1100,1200,1300,...,4500,4600,5100,5110,5111,5120,5121,5200,5300,6000
35,Australia,2000,8.21,30.377,10.687945,6.890812,48.5,11.471,6.152,0.0,...,0.0,0.0,7.957,3.658,3.379,4.299,2.800,0.769,0.0,0.0
36,Australia,2001,7.57,28.890,10.638735,7.065841,48.5,11.915,4.349,0.0,...,0.0,0.0,8.012,3.736,3.631,4.276,2.717,0.766,0.0,0.0
37,Australia,2002,7.93,29.739,10.605550,6.817096,48.5,11.773,4.949,0.0,...,0.0,0.0,8.271,4.011,3.899,4.260,2.700,0.776,0.0,0.0
38,Australia,2003,8.12,29.926,10.742314,6.580981,48.5,11.891,4.960,0.0,...,0.0,0.0,8.115,4.074,3.957,4.041,2.539,0.832,0.0,0.0
39,Australia,2004,8.42,30.258,10.917760,6.653405,48.5,12.143,5.468,0.0,...,0.0,0.0,7.879,3.970,3.864,3.909,2.476,0.808,0.0,0.0


In [23]:
X_lr = data_lr.drop(['Country', 'Year', '1100', 'Top_1', '6000'], axis=1)
X_lr.head()

,Tax_Rev_to_GDP,Individual_Spending_Rate,Society_Spending_Rate,Top_Rate,1200,1300,2100,2200,2300,2400,...,4400,4500,4600,5100,5110,5111,5120,5121,5200,5300
35,30.377,10.687945,6.890812,48.5,6.152,0.0,0.0,0.0,0.0,0.0,...,1.381,0.0,0.0,7.957,3.658,3.379,4.299,2.800,0.769,0.0
36,28.890,10.638735,7.065841,48.5,4.349,0.0,0.0,0.0,0.0,0.0,...,1.282,0.0,0.0,8.012,3.736,3.631,4.276,2.717,0.766,0.0
37,29.739,10.605550,6.817096,48.5,4.949,0.0,0.0,0.0,0.0,0.0,...,1.372,0.0,0.0,8.271,4.011,3.899,4.260,2.700,0.776,0.0
38,29.926,10.742314,6.580981,48.5,4.960,0.0,0.0,0.0,0.0,0.0,...,1.496,0.0,0.0,8.115,4.074,3.957,4.041,2.539,0.832,0.0
39,30.258,10.917760,6.653405,48.5,5.468,0.0,0.0,0.0,0.0,0.0,...,1.260,0.0,0.0,7.879,3.970,3.864,3.909,2.476,0.808,0.0


In [24]:
y_lr = data_lr.Top_1
y_lr.head()

35    8.21
36    7.57
37    7.93
38    8.12
39    8.42
Name: Top_1, dtype: float64

### Modeling

In [25]:
#Split data into 2: 75% train, 25% test
X_train, X_test, y_train, y_test = train_test_split(X_lr, y_lr, test_size=0.25, random_state=2018)

In [26]:
lr = LinearRegression()
# Fit the model against the training data
lr.fit(X_train, y_train)
# Evaluate the model against the testing data
print(mean_squared_error(y_test, lr.predict(X_test)))

5.558174547666281


/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


### Lasso

In [43]:
lasso = linear_model.Lasso(alpha=1)

lasso.fit(X_train, y_train)
# Evaluate the model against the testing data
print(lasso.score(X_test, y_test))

test_pred = lasso.predict(X_test)
print(mean_squared_error(y_test, test_pred))


0.17860974055122703
7.862441162419999


In [28]:
# These are the coefficients when it refit using that best alpha
list(zip(X_train.columns,lasso.coef_))

[('Tax_Rev_to_GDP', 0.07743572282228257),
 ('Individual_Spending_Rate', -0.8171190504444836),
 ('Society_Spending_Rate', 0.18052591256163011),
 ('Top_Rate', 0.01451069171598964),
 ('1200', -0.19245259629828346),
 ('1300', -1.4130320403368064),
 ('2100', 0.5919191990828333),
 ('2200', 0.14402451704190938),
 ('2300', -1.6966303122427233),
 ('2400', -25.52465658324905),
 ('4100', 1.3802860364522076),
 ('4200', -1.0604981294758309),
 ('4300', -3.2871488660269916),
 ('4400', -0.2020198924608801),
 ('4500', 6.875411880430372),
 ('4600', 39.70321585944909),
 ('5100', 4.497469684884132),
 ('5110', -4.120282508527244),
 ('5111', -0.6839318687853967),
 ('5120', -4.345565113228257),
 ('5121', 1.418840087074524),
 ('5200', -1.554563842843764),
 ('5300', -7.145060432509672)]

In [51]:
# Run the cross validation, find the best alpha, refit the model on all the data with that alpha

alphavec = 10**np.linspace(-6,2,90)

lasso3 = LassoCV(alphas = alphavec, cv=5)
lasso3.fit(X_train,y_train)

/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: Conv

/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: Conv

LassoCV(alphas=array([1.00000e-06, 1.22995e-06, ..., 8.13041e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [52]:
lasso3.alpha_

1e-06

In [53]:
# These are the coefficients when it refit using that best alpha
list(zip(X_train.columns,lasso3.coef_))

[('Tax_Rev_to_GDP', 0.07946272435029529),
 ('Individual_Spending_Rate', -0.8187414434450987),
 ('Society_Spending_Rate', 0.17987336557880299),
 ('Top_Rate', 0.014281730090703859),
 ('1200', -0.1953367285467624),
 ('1300', -1.4190380082802385),
 ('2100', 0.5946346955935956),
 ('2200', 0.14126331033689163),
 ('2300', -1.6998411892920313),
 ('2400', -25.47232558352683),
 ('4100', 1.3712457758005647),
 ('4200', -1.0559414284048334),
 ('4300', -3.2816287453156736),
 ('4400', -0.2022556371309546),
 ('4500', 6.875907033752111),
 ('4600', 39.52138595123029),
 ('5100', 0.24352913615804792),
 ('5110', 0.12708491797064442),
 ('5111', -0.6802808903165641),
 ('5120', -0.09591470211766724),
 ('5121', 1.4143138182295716),
 ('5200', -1.5739035307577969),
 ('5300', -7.140177852380378)]

In [54]:
# print(lr_model8.score(X_te, y_test))
test_pred = lasso3.predict(X_test)
print(mean_squared_error(y_test, test_pred))

5.530438348642651


### Try XGBoost

In [55]:
X.head()

,Tax_Rev_to_GDP,Individual_Spending_Rate,Society_Spending_Rate,Top_Rate,1200,1300,2100,2200,2300,2400,...,4400,4500,4600,5100,5110,5111,5120,5121,5200,5300
0,20.620,NaN,NaN,NaN,3.357,0.0,0.0,0.0,0.0,0.0,...,0.402,0.0,0.0,6.189,1.519,0.0,4.671,3.205,0.969,0.0
1,19.840,NaN,NaN,NaN,2.971,0.0,0.0,0.0,0.0,0.0,...,0.401,0.0,0.0,5.894,1.403,0.0,4.492,3.107,0.946,0.0
2,20.419,NaN,NaN,NaN,2.946,0.0,0.0,0.0,0.0,0.0,...,0.445,0.0,0.0,5.985,1.429,0.0,4.557,3.059,0.966,0.0
3,20.349,NaN,NaN,NaN,3.177,0.0,0.0,0.0,0.0,0.0,...,0.535,0.0,0.0,5.875,1.509,0.0,4.366,2.881,0.956,0.0
4,20.691,NaN,NaN,NaN,3.246,0.0,0.0,0.0,0.0,0.0,...,0.607,0.0,0.0,5.748,1.543,0.0,4.205,2.654,0.911,0.0


In [56]:
y.head()

0    6.34
1    6.16
2    6.25
3    6.05
4    5.92
Name: Top_1, dtype: float64

In [57]:
#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2018)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2019)

In [40]:
eval_set=[(X_train,y_train),(X_val,y_val)]

In [41]:
gbm = xgb.XGBRegressor( 
                       n_estimators=30000, #arbitrary large number
                       max_depth=3,
                       objective="reg:linear",
                       learning_rate=.1, 
                       subsample=1,
                       min_child_weight=1,
                       colsample_bytree=.8
                      )

In [ ]:
fit_model = gbm.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    #eval_metric='rmse',
                    early_stopping_rounds=50,
                    verbose=True #gives output log as below
                   )

In [55]:
test_pred = gbm.predict(X_test)

In [56]:
print(mean_squared_error(y_test, test_pred))

3.080550812179901


In [58]:
print(r2_score(y_test, test_pred))

0.8258982155916544


In [62]:
with open('gbm_initial.pkl', 'wb') as f:
    pickle.dump(gbm, f)